In [0]:
from pyspark.sql.functions import current_timestamp
from delta.tables import DeltaTable

In [0]:
def add_ingestion_date(input_df, alias=None):
    if alias is None:
        output_df = input_df.withColumn("ingestion_date", current_timestamp())
    else:
        output_df = input_df.withColumn(f"{alias}_ingestion_date", current_timestamp())
    return output_df

In [0]:
def merge_delta_data(input_df, db_name, table_name, merge_condition, partition_column):
    spark.conf.set("spark.databricks.optimizer.dynamicPartitionPruning","true")

    if spark._jsparkSession.catalog().tableExists(f"{db_name}.{table_name}"):
        deltaTable = DeltaTable.forName(spark, f"{db_name}.{table_name}")
        deltaTable.alias("tgt").merge(
            input_df.alias("src"),
            merge_condition) \
          .whenMatchedUpdateAll() \
          .whenNotMatchedInsertAll() \
          .execute()
    else:
        input_df.write.mode("overwrite").partitionBy(partition_column).format("delta").saveAsTable(f"{db_name}.{table_name}")